In [27]:
import efinance as ef
import plotly.express as px
import time
import pandas as pd

In [ ]:
#ef.api.stock.config.EASTMONEY_STOCK_BASE_INFO_FIELDS
#ef.api.stock.config.EASTMONEY_STOCK_DAILY_BILL_BOARD_FIELDS
#ef.api.stock.config.HERE
#print(ef.stock.get_members.__doc__)

## market-level command

In [ ]:
# get dates of available financial reports
all_filings_dates = ef.stock.get_all_report_dates()

In [ ]:
#print(ef.stock.get_all_company_performance.__doc__)
# get data of all stocks for a given fiscal quarter
all_financials_df = ef.stock.get_all_company_performance()
all_financials_df.head()

## 个股级别数据

In [7]:
ticker = '601888'  # 中国中免

In [15]:
stock_base_info = ef.stock.get_base_info(ticker)  # 基本信息

In [18]:
stock_name = stock_base_info.get("股票名称")

### 资金流向数据

In [44]:
# 龙虎榜
ef.stock.get_daily_billboard(start_date='2024-01-10', end_date='2024-01-14')

,股票代码,股票名称,上榜日期,解读,收盘价,涨跌幅,换手率,龙虎榜净买额,龙虎榜买入额,龙虎榜卖出额,龙虎榜成交额,市场总成交额,净买额占总成交比,成交额占总成交比,流通市值,上榜原因
0,000017,深中华A,2024-01-12,普通席位买入，成功率55.69%,6.55,10.0840,1.8610,1.421804e+07,3.182822e+07,1.761018e+07,4.943839e+07,77702715,18.297994,63.625051,2.285071e+09,连续三个交易日内，涨幅偏离值累计达到20%的证券
1,000628,高新发展,2024-01-12,主力做T，成功率42.37%,45.21,-8.0537,30.7320,-3.364789e+08,2.073377e+08,5.438166e+08,7.511543e+08,2725567038,-12.345280,27.559562,8.683667e+09,日跌幅偏离值达到7%的前5只证券
2,000638,万方发展,2024-01-12,2家机构买入，成功率33.09%,7.69,10.0143,15.7687,3.859693e+07,7.187010e+07,3.327318e+07,1.051433e+08,349343178,11.048427,30.097419,2.380452e+09,日涨幅偏离值达到7%的前5只证券
3,000676,智度股份,2024-01-12,2家机构买入，成功率42.15%,8.66,10.0381,21.3258,8.531008e+07,2.923859e+08,2.070758e+08,4.994617e+08,2311985347,3.689906,21.603150,1.103998e+10,日涨幅偏离值达到7%的前5只证券
4,000676,智度股份,2024-01-12,1家机构买入，成功率42.17%,8.66,10.0381,21.3258,6.822072e+07,3.158509e+08,2.476302e+08,5.634810e+08,2611855045,2.611964,21.573978,1.103998e+10,连续三个交易日内，涨幅偏离值累计达到20%的证券
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191,831906,舜宇精工,2024-01-10,实力游资卖出，成功率25.05%,22.79,-5.4357,54.7197,-1.521503e+07,2.040871e+07,3.562375e+07,5.603246e+07,244840400,-6.214265,22.885299,4.549887e+08,当日换手率达到20%的前5只股票
192,836826,盖世食品,2024-01-10,西藏自治区资金买入，成功率30.89%,12.79,13.0858,40.3679,-1.216123e+07,2.104933e+07,3.321056e+07,5.425990e+07,253881200,-4.790126,21.372161,6.507711e+08,当日换手率达到20%的前5只股票
193,838262,太湖雪,2024-01-10,实力游资买入，成功率40.25%,19.33,10.9007,38.7602,-7.157527e+05,1.144917e+07,1.216492e+07,2.361408e+07,113147900,-0.632582,20.870103,2.995556e+08,当日换手率达到20%的前5只股票
194,873132,泰鹏智能,2024-01-10,主力做T，成功率13.68%,22.53,1.0314,64.0556,-8.768590e+06,1.483739e+07,2.360598e+07,3.844337e+07,180569000,-4.856088,21.290126,2.569069e+08,当日价格振幅达到30%的前5只股票


In [72]:
flow_renamer = {
    '主力净流入': "prime_order_net_inflow", 
    '超大单净流入': "super_order_net_inflow", 
    '大单净流入': "big_order_net_inflow", 
    '中单净流入': "medium_order_net_inflow", 
    '小单净流入': "small_order_net_inflow",
    "时间": "time"
}

In [80]:
# 个股分钟级资金流向
flow_min = ef.stock.get_today_bill(ticker)\
    .rename(columns=flow_renamer)\
    .set_index('time')\
    .loc[:, ["prime_order_net_inflow", "super_order_net_inflow", 
             "big_order_net_inflow", "medium_order_net_inflow", 
             "small_order_net_inflow"]]
flow_min.head()

,prime_order_net_inflow,super_order_net_inflow,big_order_net_inflow,medium_order_net_inflow,small_order_net_inflow
time,,,,,
2024-01-15 09:31,-4173580.0,-3537478.0,-636102.0,1547855.0,2633801.0
2024-01-15 09:32,-3813492.0,1377881.0,-5191373.0,1296188.0,2533502.0
2024-01-15 09:33,-3684168.0,1765025.0,-5449193.0,616259.0,2930431.0
2024-01-15 09:34,-4363409.0,672094.0,-5035503.0,-968722.0,5308013.0
2024-01-15 09:35,-5376431.0,1147044.0,-6523475.0,-1162412.0,6554964.0


In [85]:
quote_renamer = {
    '日期': 'time', '开盘':'open', '收盘': 'close', '最高': 'high', '最低': 'low', 
    '成交量': 'volume', '成交额':'price*volume', '换手率': 'turnover',
    '涨跌幅': 'change_pct', '涨跌额': 'change'
}

In [88]:
quote_min = ef.stock.get_quote_history(
    stock_codes=ticker,
    beg='20240115',
    klt=1,
    fqt=0,
).rename(columns=quote_renamer).set_index('time').drop(columns=['股票名称','股票代码','振幅'])

In [96]:
intraday_data = pd.concat([quote_min, flow_min], axis=1).rename(pd.to_datetime).assign(min_into_day=lambda df: [dt.hour * 60 + dt.minute for dt in df.index])

In [99]:
# scatter plot of intraday data
xvar = "prime_order_net_inflow"
yvar = "change_pct"
cvar = 'min_into_day'
hvars = None
fig = px.scatter(intraday_data, x=xvar, y=yvar, color=cvar, hover_data=hvars)
# update layout, set title, labels, and figsize
fig.update_layout(
    title=f"{stock_name} {xvar} vs {yvar}",
    xaxis_title=xvar,
    yaxis_title=yvar,
    width=800,
    height=600,
    template="plotly_dark" 
)
fig.show()

In [64]:
# 给个股资金流向作图
fig = px.line(flow_min, x='时间', y=['超大单净流入', '大单净流入', '中单净流入', '小单净流入'])
fig.update_layout(
    title='{0}({1})分钟级资金流向'.format(stock_name, ticker), 
    xaxis_title='时间', yaxis_title='净流入', 
    template='plotly_dark')
fig.show()

fig2 = px.line(quote_min, x='日期', y=['收盘'])
fig2.show()

### 批量实时行情数据
可提供全市场最新的行情数据，速度较快，可供滚动抓取。

In [38]:
{
    None: "沪深京A股市场行情",
    "沪深A股": "沪深A股市场行情",
    "沪A": "沪市A股市场行情",
    "深A": "深市A股市场行情",
    "北A": "北证A股市场行情",
    "可转债": "沪深可转债市场行情",
    "期货": "期货市场行情",
    "创业板": "创业板市场行情",
    "美股": "美股市场行情",
    "港股": "港股市场行情",
    "中概股": "中国概念股市场行情",
    "新股": "沪深新股市场行情",
    "科创板": "科创板市场行情",
    "沪股通": "沪股通市场行情",
    "深股通": "深股通市场行情",
    "行业板块": "行业板块市场行情",
    "概念板块": "概念板块市场行情",
    "沪深系列指数": "沪深系列指数市场行情",
    "上证系列指数": "上证系列指数市场行情",
    "深证系列指数": "深证系列指数市场行情",
    "ETF": "ETF 基金市场行情",
    "LOF": "LOF 基金市场行情"
}

# please reformat the dict above
ef.stock.get_realtime_quotes('美股')

,股票代码,股票名称,涨跌幅,最新价,最高,最低,今开,涨跌额,换手率,量比,动态市盈率,成交量,成交额,昨日收盘,总市值,流通市值,行情ID,市场类型,更新时间,最新交易日
0,CRGE,Charge Enterprises Inc,187.5,0.23,0.34,0.11,0.11,0.15,214.44,226.31,-1.12,461127376,112772574.0,0.08,49459170,49459170,105.CRGE,美股,2024-01-12 13:00:00,2024-01-12
1,OTECW,OceanTech Acquisitions I Corp W,133.33,0.07,0.07,0.07,0.07,0.04,-,-,-,110,8.0,0.03,-,-,105.OTECW,美股,2024-01-12 13:00:00,2024-01-12
2,BLUA_WS,BlueRiver Acquisition Corp Wt,133.33,0.07,0.07,0.05,0.05,0.04,-,1.5,-,72436,3784.0,0.03,-,-,107.BLUA_WS,美股,2024-01-12 13:00:00,2024-01-12
3,VSACW,Vision Sensing Acquisition Corp,100.0,0.02,0.02,0.02,0.02,0.01,-,0.01,-,520,9.0,0.01,-,-,105.VSACW,美股,2024-01-12 13:00:00,2024-01-12
4,TRONW,Corner Growth Acquisition Corp,100.0,0.04,0.04,0.03,0.03,0.02,-,0.04,-,2059,65.0,0.02,-,-,105.TRONW,美股,2024-01-12 13:00:00,2024-01-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11676,XBPEW,XBP Europe Holdings Inc Wt,-50.0,0.04,0.07,0.04,0.07,-0.04,-,1.16,-,8505,583.0,0.08,-,-,105.XBPEW,美股,2024-01-12 13:00:00,2024-01-12
11677,OPA_WS,Magnum Opus Acquisition Ltd Wt,-50.0,0.01,0.02,0.01,0.02,-0.01,-,8.59,-,10400,145.0,0.02,-,-,106.OPA_WS,美股,2024-01-12 13:00:00,2024-01-12
11678,KRNLW,Kernel Group Holdings Inc Wt,-50.0,0.03,0.03,0.03,0.03,-0.03,-,0.6,-,13890,426.0,0.06,-,-,105.KRNLW,美股,2024-01-12 13:00:00,2024-01-12
11679,AWINW,AERWINS Technologies Inc Wt,-50.0,0.01,0.02,0.01,0.01,-0.01,-,0.82,-,5699,99.0,0.02,-,-,105.AWINW,美股,2024-01-12 13:00:00,2024-01-12


### 个股实时行情数据
包含成交量价，以及订单簿数据，每30秒更新一次。

In [28]:
# get the quote snapshot every 5 seconds for 10 times and concatenate them into a dataframe
dflist = []
while True:
    df = ef.stock.get_quote_snapshot(ticker)
    dflist.append(df)
    time.sleep(5)
    if len(dflist) == 10:
        break
df = pd.concat(dflist, axis=1)

In [33]:
ef.stock.get_belong_board(ticker)

,股票名称,股票代码,板块代码,板块名称,板块涨幅
0,中国中免,601888,BK0485,旅游酒店,3.87
1,中国中免,601888,BK0150,北京板块,-0.52
2,中国中免,601888,BK0927,免税概念,1.29
3,中国中免,601888,BK0665,电商概念,0.61
4,中国中免,601888,BK0499,AH股,0.29
5,中国中免,601888,BK0718,证金持股,0.27
6,中国中免,601888,BK0505,中字头,0.12
7,中国中免,601888,BK0879,标准普尔,-0.01
8,中国中免,601888,BK0611,上证50_,-0.06
9,中国中免,601888,BK0683,国企改革,-0.08


### 个股/批量历史行情
1. 可精细到分钟级
2. 提供复权选项

In [37]:
ef.stock.get_quote_history(
    stock_codes='NVDA',
    klt=101,
    fqt=0,
)

,股票名称,股票代码,日期,开盘,收盘,最高,最低,成交量,成交额,振幅,涨跌幅,涨跌额,换手率
0,英伟达,NVDA,1999-01-22,21.00,19.69,23.44,18.62,67867200,0.000000e+00,40.17,64.08,7.69,237.33
1,英伟达,NVDA,1999-01-25,21.25,21.75,22.00,19.69,12762000,0.000000e+00,11.73,10.46,2.06,44.63
2,英伟达,NVDA,1999-01-26,22.00,20.06,22.44,19.75,8580000,0.000000e+00,12.37,-7.77,-1.69,30.00
3,英伟达,NVDA,1999-01-27,20.12,20.00,20.62,19.00,6109200,0.000000e+00,8.08,-0.30,-0.06,21.36
4,英伟达,NVDA,1999-01-28,20.00,19.94,20.12,19.81,5688000,0.000000e+00,1.55,-0.30,-0.06,19.89
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5090,英伟达,NVDA,2024-01-08,495.12,522.53,522.75,494.79,64250990,3.298366e+10,5.69,6.43,31.56,2.61
5091,英伟达,NVDA,2024-01-09,524.01,531.40,543.25,516.90,77310006,4.108637e+10,5.04,1.70,8.87,3.14
5092,英伟达,NVDA,2024-01-10,536.16,543.50,546.00,534.89,53379576,2.891926e+10,2.09,2.28,12.10,2.16
5093,英伟达,NVDA,2024-01-11,549.99,548.22,553.46,535.60,59675877,3.257517e+10,3.29,0.87,4.72,2.42


## 指数数据

In [8]:
index_ticker = {
    'CSI300': '000300',  # 沪深300
    'CSI1000': '000852',  # 中证1000
}

In [ ]:
# 获取指数成分股
index_members = ef.stock.get_members(index_ticker['CSI300'])  # 沪深300成分股

In [ ]:
# 通过并行计算，获取沪深300成分股的基本信息
index_const_base_info=ef.stock.get_base_info(index_members['股票代码'].values.tolist())